In [1]:
import datetime

import time

import sched

import sys
import os
import pickle

import pandas as pd
import re
import ast

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup

In [2]:
def format_date(date_n_time):

    r""" Format datetime.datetime.now() output my way.
    
    Parameters
    ----------
    date_n_time : datetime.datetime.now() output
    
    Returns
    -------
                : str 
        datetime.datetime.now() formatted my way
    """
    
    return 'DAY_'  + str(date_n_time.month) + '-' + str(date_n_time.day) + '-' + str(date_n_time.year) \
           + '_TIME_' + str(date_n_time.hour) + '_' + str(date_n_time.minute)

In [3]:
def drop_leading_zeros_from_day_part(month_n_day):
    
    r"""Takes a date of form month-day and 
    drops the leading zero from the day if it is there. 
    Ex: 10-29-> 10-29 and 10-05 -> 10-8
    This formats the dates the same throughout.
    
    Parameters
    ----------
    month_n_day : str
         
    Returns
    -------
                : str
    
    """    
    
    month_n_day = month_n_day.split('-')
    
    if month_n_day[1][0]=='0':
        month_n_day[1] = month_n_day[1][1]
        
    return ('-').join(month_n_day)

In [4]:
def format_accuweather_daily_weather_forecast_soup(soup_in):
    r"""Takes a Beautiful soup object from Accuweather.com's Daily Weather Forecast page (and only that page style)
    and returns a dictionary with predictions for that day and night. 
    Notes: 
        This is not pretty, but I don't see how else to do it. 
        (Regex doesn't seem helpful bc it is easy to screw up.) 

    Parameters
    ----------
    soup_in : Beautiful soup object
        Beautiful soup object from Accuweather.com's Daily Weather Forecast page
    
    Returns
    -------
    final_dict: dictionary
        dictionary of weather prediction information 
    
    """
    
    final_dict = {}
    
    # grab the information of interest, and split it into day and night info
    soup_day_n_night = soup_in.find_all('div',{ 'id':"detail-day-night"})[0]    
    soup_day = soup_day_n_night.find_all('div',{"class":"day"})[0]
    soup_night = soup_day_n_night.find_all('div',{"class":"night"})[0]
    
    # get daytime info
    final_dict['hi_temp_day'] = soup_day.find('span',{'class':"large-temp"}).text
    final_dict['realfeel_day'] = soup_day.find('span',{'class':"realfeel"}).text
    final_dict['precip_chance_day'] = soup_day.find('span',{'class':"precip"}).text
    final_dict['condns_day'] = soup_day.find('div',{'class':"cond"}).text.lstrip().rstrip()  
    
    wind_info_day  = [line.strip() for line in soup_day.find('ul',{'class':"wind-stats"}).text.splitlines() if line is not '']    
    final_dict['wind_speed_day'] = [i for i in wind_info_day if 'mph' in i and 'Gusts' not in i][0]
    final_dict['wind_gusts_day'] = [i for i in wind_info_day if 'mph' in i and 'Gusts' in i][0]
    
    stats_day = [line for line in soup_day.find('ul',{'class':"stats"}).text.split('\n') if line is not '']   
    final_dict['uv_index_day'],\
    final_dict['thunderstorms_prob_day'], \
    final_dict['total_precip_amt_day'], \
    final_dict['rain_amt_day'], \
    final_dict['snow_amt_day'], \
    final_dict['ice_amt_day'], \
    final_dict['hours_of_precip_day'], \
    final_dict['hours_of_rain_day'] = [x.split(": ")[1] for x in stats_day]
    
    # get nightime info

    # this isn't a typo, they call it 'large-temp' but it is the low for the period (night)
    final_dict['low_temp_night'] = soup_night.find('span',{'class':"large-temp"}).text
    final_dict['realfeel_night'] = soup_night.find('span',{'class':"realfeel"}).text
    final_dict['precip_chance_night'] = soup_night.find('span',{'class':"precip"}).text
    final_dict['condns_night'] = soup_night.find('div',{'class':"cond"}).text.lstrip().rstrip()  
    
    wind_info_night  = [line.strip() for line in soup_night.find('ul',{'class':"wind-stats"}).text.splitlines() if line is not '']    
    final_dict['wind_speed_night'] = [i for i in wind_info_night if 'mph' in i and 'Gusts' not in i][0]
    final_dict['wind_gusts_night'] = [i for i in wind_info_night if 'mph' in i and 'Gusts' in i][0]
    
    stats_night = [line for line in soup_night.find('ul',{'class':"stats"}).text.split('\n') if line is not '']       
    final_dict['uv_index_night'],\
    final_dict['thunderstorms_prob_night'], \
    final_dict['total_precip_amt_night'], \
    final_dict['rain_amt_night'], \
    final_dict['snow_amt_night'], \
    final_dict['ice_amt_night'], \
    final_dict['hours_of_precip_night'], \
    final_dict['hours_of_rain_night'] = [x.split(": ")[1] for x in stats_night]   
        
    # just to keep this handy in original(-ish) format
    temp_raw_text = [line.strip() for line in soup_in.find_all('div',{ 'id':"detail-day-night"})[0].text.split("\n") \
                     if line.strip() is not '']
    temp_raw_text = ' \n '.join(temp_raw_text)
    
    final_dict['raw_text'] = temp_raw_text
    
    return final_dict

In [5]:
acw_default_base_sauce_default = 'https://www.accuweather.com/en/us/lebanon-nh/ZIP/daily-weather-forecast/334463?day='
# accuweather requires acting more like an actual browser
acw_headers_default = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'}

def get_accuweather_daily_forecast(day,
                                   zip_code,
                                   acw_base_sauce = acw_default_base_sauce_default,
                                   awc_headers = acw_headers_default):    

    r""" Returns a daily forecast from accuweather.com
    
    Parameters
    ----------
    day : int
        Day for forecast. Today is 1, tomorrow is 2, etc...
    zip_code: str
        
    acw_base_sauce : str
        'base' url for accuweather.com that needs the day above appended.  
    awc_headers : 1 element dictionary.
        Dictionary to feed to requests to fake a browser otherwise request is denied. 
        Could probably do this with selenium too. 
    
    Returns
    -------
    acw_forecast_for_date: str   
        Date string of day forecast is for.
    acw_weather_dict: dictionary
        Forecast in terms of dictionary. 
    """
    
    total_sauce = acw_base_sauce.replace('ZIP', zip_code) + str(day)
    response = requests.get(total_sauce, headers = awc_headers)
    acw_soup = BeautifulSoup(response.text,'html.parser')

    # The date is in in the first row, last column of the only table.
    acw_forecast_for_date = acw_soup.find_all('tr')[0].find_all('th')[-1].text
    acw_forecast_for_date = acw_forecast_for_date.strip().replace('/','-')
    
    # weird but accuweather has the wrong year in its friggin dates 
    # dropping this for now, may add it back in
#    this_year = str(datetime.datetime.now().year)
    acw_year = acw_forecast_for_date[-4:]
    
#     if acw_year != this_year:
#         acw_forecast_for_date = acw_forecast_for_date.replace(acw_year,this_year)

    acw_weather_dict = format_accuweather_daily_weather_forecast_soup(acw_soup)

    del response, acw_soup
    
    return acw_forecast_for_date, acw_weather_dict

In [6]:
def get_accuweather_X_day_forecast(to_day, zip_code, from_day = 1, pause_for = 2):
    
    r"""Returns accuweather forecast between from_day and to_day.
    
    Parameters
    ----------
    to_day : int
        Day to stop getting forecasts, including that day. 
    zip_code : str    
    from_day : int 
        Day to start getting forecasts. Today is 1, tomorrow is 2 etc...
    pause_for : int
        Pause this many secs between forecast requests so we don't slam website.
        
    Returns
    -------
    master_acw_dict
        Dictionary with entries 'date':'forecast for that date' 
    
    """
    
    print("\n")
    print("Getting accuweather.com forecast")
    
    master_acw_dict = {}

    for d in range(from_day, to_day + 1):
        acw_key, acw_forecast = get_accuweather_daily_forecast(day = d, zip_code = zip_code)
        # drop year 
        acw_key = re.findall('\d+-\d+', acw_key)[0]
        
        master_acw_dict[acw_key] = acw_forecast
        # Just so we don't slam the website
        time.sleep(pause_for)
    
    return master_acw_dict

In [7]:
wdc_base_url = 'https://weather.com/weather/monthly/l/03766'

def get_weatherdcom_15day_forecast(zip_code, wdc_sauce = wdc_base_url):

    r"""Gets 15 day forecast from weather.com's monthly (or 10 day weather) page (and only those pages)
    and returns a dictionary with predictions for 15 days. 
    Notes: 

    Parameters
    ----------
    zip_code : str
    wdc_sauce : str
        url for weather.com's monthly Forecast page
    
    Returns
    -------
    dictionary of dictionaries
        dictionary of weather prediction information 
        
    NOTE THAT:
    (0) 10 day and monthly both work from the same dictionary. 
    Two weird things about weather.com:
    (1) On their 10 day they actually give a 15 day forecast
    (2) There is more information in the html file than is displayed in browser!!
    (3) This essentially comes from the website as a dictionary, so no need to format right now. 
    """
    
    print("\n")
    print("Getting weather.com forecast")
    
    wdc_sauce = wdc_base_url.replace('ZIP', zip_code)
    response = requests.get(url = wdc_sauce)
    wdc_soup = BeautifulSoup(response.text,"lxml")

    # this regex marks the forecast in the html
    search_for = '"vt1dailyForecast":\[(.*?)\]'
    fifteen_day_forecast = re.findall(search_for, wdc_soup.text)
    # this has to be length 1 or else something is wrong
    if len(fifteen_day_forecast)==1:
        print("Correct length == 1 list from weather.com recieved.")
    if len(fifteen_day_forecast)!=1:
        print("Incorrect length != 1 list from weather.com received.")
    
    # format the str a bit so I can make it a literal
    fifteen_day_forecast_formatted = '[' + fifteen_day_forecast[0] + ']'
    fifteen_day_forecast_formatted = fifteen_day_forecast_formatted.replace('null','"null"')
    wdc_list_of_dicts = ast.literal_eval(fifteen_day_forecast_formatted)
    
    wdc_dict_of_dicts = {}
    for d in wdc_list_of_dicts:
        date_n_time = d.pop('validDate')
        just_date = date_n_time.split('T')[0]
        # drop the year
        just_date = just_date[-5:]
        
        just_date = drop_leading_zeros_from_day_part(just_date)
        
        wdc_dict_of_dicts[just_date] = d
    
    return wdc_dict_of_dicts  

In [8]:
# mismatched state and zip codes don't always give an error, sometimes give a real city. 
# state is lower case 
wugd_default_base_url = 'https://api-ak.wunderground.com/us/STATE/ZIP/'

def get_weather_underground_10day(state, zip_code, wugd_base_sauce = wugd_default_base_url):
    r"""
    Takes a specific weather underground site and returns 10 day forcast. 
    This site has a lot of danmic content and so requires very careful clicking 
    hence more code and error trapping. 
    
    Parameters
    ----------
    state: str
        Lower case 2 digit state code
    zip_code: str
        5 digit zip code 
    wugd_base_sauce: str
        http address

    Returns
    -------
    List of weather underground 10 day forecasts.
    
    """
    
    print("\n")
    print("Getting weatherunderground.com forecast")
    
    # open the browser and scroll to bottom to update full page (may want to just scroll to forecast tabs?)
    browser = webdriver.Chrome()
    final_sausce = wugd_base_sauce.replace('ZIP', zip_code)
    browser.get(final_sausce)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # try to click the 'description tab'
    try_count = 0
    while True:   
        try_count += 1
        try:
            descrip_tab = browser.find_element_by_id("forecast-tab-descriptive")

            # this attribute seems to correlate with whether or not decription tab is fully loaded. 
            descp_tab_role = descrip_tab.get_attribute("role")
            if descp_tab_role == None:
                print("Description tab doesn't exist yet")
                raise ValueError
            elif descp_tab_role=='presentation':
                print("Description tab is in presentation mode")
            elif descp_tab_role != None and descp_tab_role!='presentation':
                print("Description tab is neither 'None' or 'presentation'...never seen that")
           
        # if 'description tab' is not loaded, wait and try again
        except (NoSuchElementException, ValueError):
            print('Waiting for weatherunderground "Forecast Description Tab" to load...')
            if try_count==10:
                print('Tried 10 times to click on "Forecast Description Tab". Something is wrong. Breaking...')
                return None
                
            time.sleep(10)
        
        # if and when it is actually loaded, click it. 
        else:
            descrip_tab.click()
            print('"Forecast Description Tab" button clicked.')
            break   
    
    # try to click the 'more days tab'
    try_count = 0
    while True:
        try_count += 1
        try:
            more_days_button = browser.find_element_by_class_name("fct3-toggle-show")
        except NoSuchElementException:
            print('Waiting for weatherunderground "More Days" to load...')
            time.sleep(10)
            if try_count==10:
                print('Tried 10 times to click on "More Days". Something is wrong. Breaking...')
                break
        else:
            more_days_button.click()
            print('"More days" button clicked.')
            break

    # get forecast and make sure all 10 days are being retrieved (final list has len == 70) 
    try_count = 0
    while True:
        try_count+=1
        wug_10_day_forecast_element = browser.find_element_by_xpath('//*[@id="extended-forecast"]')
        wug_10_day_forecast = wug_10_day_forecast_element.text.split("\n")
        if len(wug_10_day_forecast) == 70:
            print('"More days" succesfully loaded.')
            break
        elif len(wug_10_day_forecast) == 21:
            print("Weather underground 'More Days' doesn't seem to have loaded yet. Waiting...")
            time.sleep(3)
        elif len(wug_10_day_forecast) != 70 and len(wug_10_day_forecast) != 21:
            print("Weather underground 'More Days' error I haven't seen yet. Breaking...")
            break
        if try_count == 10:
            print('Tried 10 times to wait for "More Days" to load. Something is wrong. Breaking...')
            break
    
    browser.close() 
    
    return wug_10_day_forecast

In [9]:
def format_weather_underground_10day(wug_10_day_forecast):
    r"""
    Takes the raw output from get_weather_underground_10day and formats it into a dict
    of dictionaries
    
    Parameters
    ----------
    wug_10_day_forecast : list of strings

    Returns
    -------
    wug_10_day_forecast_dict : Dictionary of dictionaries
    
    """
    
    if len(wug_10_day_forecast)==70:
        print('weather underground list is correct length of 70 = 10days*7elements')
    elif len(wug_10_day_forecast)!=70:
        print('weather underground list is INcorrect length, not 70.')
    
    # list comes as flattened lists of len 7 of all 10 days, so unflatten lists
    wug_10_day_forecast_split = [wug_10_day_forecast[i:i+7] for i in range(0,70,7)]    
   
    wug_10_day_forecast_dict = {}

    for forecast in wug_10_day_forecast_split:

        temps = re.findall('\d+', forecast[2])
        temps = [int(x) for x in temps]

        day_precp_chance = re.findall('\d+ %', forecast[3])[0]
        day_precip_amnt = re.findall('\d+ in', forecast[3])[0]

        night_precp_chance = re.findall('\d+ %', forecast[5])[0]
        night_precip_amnt = re.findall('\d+ in', forecast[5])[0]
        
        date = forecast[1].replace('/','-')
        
        date = drop_leading_zeros_from_day_part(date)
        
        wug_10_day_forecast_dict[date] = {'day_of_week': forecast[0],
                                          'high_temp': max(temps),
                                          'low_temp': min(temps),
                                          'day_precp_chance': day_precp_chance,
                                          'day_precip_amnt': day_precip_amnt,
                                          'day_descrip': forecast[4],
                                          'night_precp_chance': night_precp_chance,
                                          'night_precip_amnt': night_precip_amnt,
                                          'night_descrip': forecast[6]
                                          }
        
    return wug_10_day_forecast_dict

In [10]:
s_and_z = {'Leb_NH':       {'zip':'03766','state':'nh'},
           'Wapp_Falls_NY':{'zip':'12590','state':'ny'},
           'Lynd_OH':      {'zip':'44124','state':'oh'},
          }
    
def find_all_daily_forecasts(state_and_ZIP_dict_in, number_of_aw_forecasts = 17):
     
    r"""Creates a directory, gets and saves forecasts for cities for the next 10-15 days depending on
    paricular website and parameters choosen. 
    
    Parameters
    ----------
    state_and_ZIP_dict_in : dict
        dict of dicts of zip and state codes
    
    number_of_aw_forecasts : int
        Number of accuweather.com days to get. Set to ~17 for now. I do this just do that 
        I have as many as weather.com (+ padding) in case I just compare those 2 for longer 
    
    Returns
    -------
    None (all output is saved during evaluation)
    
    """      
    
    # get current time and format
    curr_time = datetime.datetime.now()
    forecast_on = format_date(curr_time)
    
    # zero out a timer in order to time a single days forecasts
    t_0 = time.time()
    
    print('Action started at ' + str(curr_time) )
    folder_date = 'Saved_on_'+ str(forecast_on) 
      
    # create the directory, sometimes when rapidly running during error trapping the directory is already there.     
    new_path = 'forecasts/' + folder_date 
    try:
        os.makedirs(new_path)
    except FileExistsError:
        print("!!Folder already exists. I likely just ran back-to-back sessions and didn't delete.")
        return None
                     
    print('folder is: ' + new_path)            
        
    # get forecast for each location, and save as pickle   
    for k in state_and_ZIP_dict_in:
        z = state_and_ZIP_dict_in[k]['zip']
        s = state_and_ZIP_dict_in[k]['state']
                
        print("Finding forecast for " + z)

        awc_forecast_dict = get_accuweather_X_day_forecast(to_day = number_of_aw_forecasts, zip_code = z)

        wdc_forecast_dict = get_weatherdcom_15day_forecast(zip_code = z)
    
        wug_forecast_raw = get_weather_underground_10day(state = s, zip_code = z)
        wug_forcast_dict = format_weather_underground_10day(wug_forecast_raw)
        
        complete_forecast = {'awc_forecast':awc_forecast_dict,
                             'wdc_forecast':wdc_forecast_dict,
                             'wug_forecast':wug_forcast_dict}
    
        with open(new_path + '/' + str(z) + '_forecast.pkl', 'wb') as handle:
            pickle.dump(complete_forecast, handle, protocol = pickle.HIGHEST_PROTOCOL)    

    just_zips = [state_and_ZIP_dict_in[k]['zip'] for k in state_and_ZIP_dict_in.keys()]  
    print("Total zip for " + str(just_zips) + " took " + str(time.time() - t_0) + " seconds." + "\n")
 
my_schedule = sched.scheduler(time.time, time.sleep)

def master_plan():
    # as far as I can tell, the delay on subsequent events is with respect to
    # the first 0, not when the prior event concludes. 
                    # delay = day*hours*min*sec
    my_schedule.enter(delay = 0*0*0*0*0,  priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 1*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 2*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 3*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 4*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 5*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 6*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 7*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 8*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 9*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 10*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 11*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 12*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 13*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 14*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 15*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 16*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 17*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 18*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 19*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 20*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 21*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 22*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 23*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 24*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 25*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 26*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 27*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 28*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 29*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.enter(delay = 30*24*60*60, priority = 1, action = find_all_daily_forecasts, kwargs = {'state_and_ZIP_dict_in':s_and_z})
    my_schedule.run()

In [ ]:
master_plan()

Action started at 2018-10-28 07:32:14.631411
folder is: forecasts/Saved_on_DAY_10-28-2018_TIME_7_32
Finding forecast for 03766


Getting accuweather.com forecast


Getting weather.com forecast
Correct length == 1 list from weather.com recieved.


Getting weatherunderground.com forecast
Description tab is in presentation mode
"Forecast Description Tab" button clicked.
Waiting for weatherunderground "More Days" to load...
"More days" button clicked.
"More days" succesfully loaded.
weather underground list is correct length of 70 = 10days*7elements
Finding forecast for 12590


Getting accuweather.com forecast


Getting weather.com forecast
Correct length == 1 list from weather.com recieved.


Getting weatherunderground.com forecast
Description tab is in presentation mode
"Forecast Description Tab" button clicked.
Waiting for weatherunderground "More Days" to load...
"More days" button clicked.
Weather underground 'More Days' doesn't seem to have loaded yet. Waiting...
"More days" succesfu

In [ ]:
ls forecasts/ 

In [ ]:
# path = 'forecasts/Saved_on_DAY_10-9-2018_TIME_9_44/'
# os.listdir(path)

In [ ]:
# with open(path + "03766_forecast.pkl",'rb') as test_file:
#     my_dict = pickle.load(test_file)

In [ ]:
#my_dict.keys()

In [ ]:
#my_dict['wug_forecast'].keys()

In [ ]:
#my_dict['awc_forecast'].keys()

In [ ]:
#my_dict['wdc_forecast'].keys()

In [ ]:
#my_dict